In [1]:
#!pip install dendropy
#!/usr/bin/python
import dendropy
from pandas import Series, DataFrame
import numpy as np
import function_1

In [11]:
#import dendropy
#from pandas import Series, DataFrame
#import numpy as np


def make_distance_matrix(distances, sort=False):
    distances = distances.reset_index().pivot_table(index=0, columns=1, values='distance')
    distances.index.name=None
    distances.columns.name=None
    distances = distances.fillna(distances.T)

    missing_rows = [i for i in distances.index if i not in distances.columns]
    for row in missing_rows:
        distances[row] = distances.loc[row]

    missing_columns = [c for c in distances.columns if c not in distances.index]
    for column in missing_columns:
        distances.loc[column] = distances[column]
    
    if sort:
        distances = distances.sort_index().sort_index(axis=1)
    return distances.fillna(0)



def d(start, end, distances):
    return distances.loc[(start, end), 'distance']



def get_distance_dataframe(tree):
    pdm = tree.phylogenetic_distance_matrix()
    distances = [[*name, distance] for name, distance in zip(pdm.distinct_taxon_pair_iter(), pdm.distances())]
    distances = DataFrame(distances)
    #print(distances)
    
    def get_label(node):
        return node.label
    
    distances[0] = distances[0].map(get_label)
    distances[1] = distances[1].map(get_label)
    
    distances = distances.set_index([0, 1])
    distances = distances.rename(columns = {2:'distance'})

    return distances


def get_distance_topology(tree):
    pdm = tree.phylogenetic_distance_matrix()
    distances = [[*name, distance] for name, distance in zip(pdm.distinct_taxon_pair_iter(), pdm.distances())]
    

    for pairs in distances:
        pairs[2] = pdm.path_edge_count(taxon1 = pairs[0], taxon2 = pairs[1])
    
    distances = DataFrame(distances)
    def get_label(node):
        return node.label
    
    distances[0] = distances[0].map(get_label)
    distances[1] = distances[1].map(get_label)
    distances = distances.set_index([0, 1])
    distances = distances.rename(columns = {2:'distance'})
    #distances = DataFrame(distances)
    
    #print("hello")
    return distances



def get_ratio(distances1, distances2, common_leaves):
    r1 = distances1.loc[common_leaves, common_leaves].sum().sum() / 2
    r2 = distances2.loc[common_leaves, common_leaves].sum().sum() / 2
    return r1/r2

def noisify_distances(distances):
    noise = np.triu(1 + np.random.random(distances.shape) / 10).round(2)
    noise += noise.T
    return distances * noise


#def temp_paralog(taxalist):
    


def get_paralogs(taxa_list1, taxa_list2):
    
    common_leaves = []
    temp_leaves = []
    paralogs_species = []
    dico_identifier = {}
    
    CommonLeaves = {}
    Paralogs = {}
    
    for c in taxa_list1:
        #This splitting method is not ideal : might not always have the same way to separate specie and gene -> add verification when launching program ?
        taxon = c.split(' ')[0]
        gene = c.split(' ')[1]
        
        if taxon in dico_identifier.keys() :
            dico_identifier[taxon].append(gene)
        else : dico_identifier[taxon] = [gene]

        if taxon in temp_leaves :
            paralogs_species.append(taxon)
            temp_leaves.remove(taxon)
            
        else :
            temp_leaves.append(taxon)

    
    for d in taxa_list2:
        taxon = d.split(' ')[0]
        gene = d.split(' ')[1]
        
        if taxon in dico_identifier.keys() :
            dico_identifier[taxon].append(gene)
        else : dico_identifier[taxon] = [gene]
        
        if taxon in temp_leaves :
            common_leaves.append(taxon)
            temp_leaves.remove(taxon)
            
        elif taxon in common_leaves :
            paralogs_species.append(taxon)
            common_leaves.remove(taxon)
    
    for specie in paralogs_species : Paralogs[specie] = dico_identifier[specie]
    for common in common_leaves : CommonLeaves[common] = dico_identifier[common]
    
    return Paralogs, CommonLeaves
            

In [12]:
def identify_CL_Para_rename(taxa1, taxa2, common_leaves, paralogs):
    
    dict_T1 = {"Specie":[], "Identifier":[], "Alias":[]}
    dict_T2 = {"Specie":[], "Identifier":[], "Alias":[]}
    
    for element in taxa1:
        data = element.label.split(' ')
        taxon = data[0]
        gene = data[1]
        dict_T1["Specie"].append(taxon)
        dict_T1["Identifier"].append(gene)
        dict_T1["Alias"].append('')
    
    for element in taxa2:
        data = element.label.split(' ')
        taxon = data[0]
        gene = data[1]
        dict_T2["Specie"].append(taxon)
        dict_T2["Identifier"].append(gene)
        dict_T2["Alias"].append('')
    
    def rename(element, dict_Tree, taxa_Tree):
        pos = 0
        for copies in range(dict_Tree["Specie"].count(element)) :
                new_ID = str(copies+1)
                pos = dict_Tree["Specie"].index(element, pos, len(dict_Tree["Specie"]))
                taxa_Tree[pos].label = element+"_"+new_ID
                dict_Tree["Alias"][pos] = new_ID
                pos += 1
        
    temp_common = np.intersect1d(dict_T1["Specie"],dict_T2["Specie"]) 
    for element in temp_common:
        
        if dict_T1["Specie"].count(element) == 1 and dict_T2["Specie"].count(element) == 1:
            taxa1[dict_T1["Specie"].index(element)].label = element
            taxa2[dict_T2["Specie"].index(element)].label = element
            common_leaves.append(element)
            
        else :
            paralogs.append(element)
            rename(element, dict_T1, taxa1)
            rename(element, dict_T2, taxa2)
    
    return dict_T1, dict_T2



def temp_distances_paralogs(matrixT1, matrixT2, common_leaves, para, df_T1, df_T2):
    # step 1 compute ratio
    #print(common_leaves)
    R = get_ratio(matrixT1, matrixT2, common_leaves)
    #print('ratio:', R)
    candidates1 = []
    candidates2 = []
    
    # step 2 compute distances from candidate to common leaves
    for copy in para :
        for i in range(len(df_T1[df_T1['Specie']==copy])):
            paralog = copy+"_"+str(i+1)
            candidates1.append(paralog)
        
        for j in range(len(df_T2[df_T2['Specie']==copy])):
            paralog = copy+"_"+str(j+1)
            candidates2.append(paralog)
    candidate_distances1 = matrixT1.loc[candidates1, common_leaves].sum(axis=1)
    candidate_distances2 = matrixT2.loc[candidates2, common_leaves].sum(axis=1)
    candidate_distances2 = candidate_distances2 * R
    print(candidate_distances1)
    print("\n")
    print(candidate_distances2)

In [4]:
taxa_1 = dendropy.TaxonNamespace()
taxa_2 = dendropy.TaxonNamespace()
tree1 = dendropy.Tree.get(path='../data/33_pruned_reconciliated.nhx', schema='newick', suppress_edge_lengths = False, taxon_namespace=taxa_1)
# or whatever relevant format if not newick
tree2 = dendropy.Tree.get(path='../data/30_pruned_reconciliated.nhx', schema='newick', suppress_edge_lengths = False, taxon_namespace=taxa_2)


Dict_T1 = {}
Dict_T2 = {}
CL = []
para = []

Dict_T1, Dict_T2 = identify_CL_Para_rename(taxa_1, taxa_2, CL, para)


In [5]:
print(tree1.as_ascii_plot())

            /----------------------------------------------------------- n3    
            |                                                                  
            |                                               /----------- n25_1 
            |                                   /-----------+                  
/-----------+                       /-----------+           \----------- n25_2 
|           |                       |           |                              
|           |           /-----------+           \----------------------- n25_3 
|           |           |           |                                          
|           |           |           |                       /----------- n37   
|           \-----------+           \-----------------------+                  
|                       |                                   \----------- n38   
+                       |                                                      
|                       |               

In [14]:
distances1 = get_distance_dataframe(tree1)
distances2 = get_distance_dataframe(tree2)

matrixT1 = make_distance_matrix(distances1, True)
matrixT2 = make_distance_matrix(distances2, True)

T1_info = DataFrame(Dict_T1, index = (Dict_T1['Specie']))
T2_info = DataFrame(Dict_T2, index = (Dict_T2['Specie']))

In [15]:
T1_info

,Specie,Identifier,Alias
n3,n3,4,
n25,n25,38,1
n25,n25,39,2
n25,n25,33,3
n37,n37,34,
n38,n38,35,
n35,n35,30,
n22,n22,36,
n40,n40,37,
n34,n34,29,


In [16]:
temp_distances_paralogs(matrixT1, matrixT2, CL, para, T1_info, T2_info)

n25_1    11.954897
n25_2    11.954897
n25_3    11.448089
n30_1    17.948169
n30_2    17.999877
dtype: float64


n25_1    13.579986
n30_1    14.634147
dtype: float64
